# Bayesian Model Identification on Lotka-Volterra System Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jonathon-langford/aims-inference-2026/blob/main/5_Bayesian_Model_Selection/project/LotkaVolterra.ipynb)

This notebook is adapted from the [Symbolic Model Workshop Notebook](https://github.com/SymbolicModel/PySINDy_Tutorial/blob/main/1_SINDy_ODE.ipynb) written by Urban Fasel (2025).

This tutorial demonstrates how to apply the principles of Bayesian model selection to discover the governing equations of the predator-prey population dynamics directly from noisy data.

The Lotka-Volterra equation is the most rudimentary equation that describes the population dynamics of predator and prey. It is defined by the following set of nonlinear ordinary differential equations:

$$
\begin{align}
\dot{x} &= \alpha x - \beta xy \\
\dot{y} &= \delta uv - \gamma y 
\end{align}
$$

Here, ${x}$ and ${y}$ are the population of prey and predator respectively, $\alpha$, $\beta$, $\gamma$, and $\delta$ are system parameters, and $\dot{x}$ and $\dot{y}$ represent the derivatives of the state variables with respect to time.


# Tutorial to warm up 
This part of this notebook is to get you warmed up for the real project. By completing this notebook, you should understand:
- how to approach the problem of learning a dynamical model from time-series data assuming the model is time-invariant
- how to approximate time-derivatives from observed data
- how to design a library of candidate functions
- how to perform regression on the data given the library
- how to select the best model
- how to evaluate and validate the model

In the real project (see below), the workflow would be similar, but the data is much more challenge and open-ended.

The tutorial is split into five steps:

- Generate noisy data: integrate the prey and predator system ODE and add Gaussian white noise
- Compute derivatives: using finite difference
- Build the library $\Theta$, here using polynomials up to order 3
- Compute Bayesian sparse regression
- Evaluate the performance of the identified model for forecasting

Make sure you have the required libraries installed, e.g. !pip install numpy scipy matplotlib scikit-learn ...

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt

from sklearn import linear_model

In [ ]:
# Seed the random number generators for reproducibility
np.random.seed(100)

# Initialize integrator keywords for solve_ivp to replicate the odeint defaults
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'RK45' #'LSODA'
integrator_keywords['atol'] = 1e-12


# ------------------------------------------------
# 1. Generate Noisy Lotka-Volterra System Data
# ------------------------------------------------

# Lotka-Volterra system parameters
alpha, beta, gamma, delta  = 4.0, 0.2, 9.0, 0.25
params = [alpha, beta, gamma, delta]

# Initial conditions x0, time step dt, and time span tspan
x0 = [4.0,30.0]
dt = 0.01
t_span = (0, 5)
t_eval = np.arange(t_span[0], t_span[1] + dt, dt)

# Lotka-Volterra system right-hand side function
def lotka_volterra_system(t, x, params):
    dx = params[0] * x[0] - params[1] * x[0] * x[1]
    dy = -params[2] * x[1] + params[3] * x[0] * x[1]
    return [dx, dy]

# Solve the initial value problem: Runga-Kutta method
solution = solve_ivp(
    fun=lotka_volterra_system,
    t_span=t_span,
    y0=x0,
    args=(params,),
    t_eval=t_eval,
    **integrator_keywords
)
x_true = solution.y.T

# Add Gaussian white noise
np.random.seed(1)
sig = 0.1
x_noisy = x_true + sig * np.std(x_true) * np.random.randn(*x_true.shape)


In [ ]:
# Plot the noisy data
plt.figure(figsize=(10, 4))
plt.plot(t_eval, x_noisy[:, 0], 'b', label='Prey (noisy)')
plt.plot(t_eval, x_noisy[:, 1], 'r', label='Predator (noisy)')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Noisy Lotka-Volterra Data')
plt.legend()

In [ ]:
# ------------------------------------------------
# 2. Compute Derivatives (finite difference)
# ------------------------------------------------

# Compute derivatives using finite difference (4th order central difference)
dx_dt = (1/(12*dt)) * (-x_noisy[5:, :] + 8*x_noisy[4:-1, :] - 8*x_noisy[2:-3, :] + x_noisy[1:-4, :])
x = x_noisy[3:-2, :] # cut tails
t = t_eval[3:-2]

In [ ]:
# ------------------------------------------------
# 3. Build library of nonlinear terms
# ------------------------------------------------

# Build the library using the pooldata function defined in utils.py

def poolData(x, polyorder):
    """Builds a polynomial library of candidate functions."""

    # If the input is a 1D array, convert it to a 2D array for consistent processing.
    if x.ndim == 1:
        x = x.reshape(1, -1)
    
    n_samples, n_vars = x.shape
    ind = 0
    # Add a column for the constant term (1)
    theta = np.ones((n_samples, 1))
    ind += 1

    # Poly order 1
    for i in range(n_vars):
        theta = np.hstack([theta, x[:, i:i+1]])
        ind += 1
    
    # Poly order 2
    if polyorder >= 2:
        for i in range(n_vars):
            for j in range(i, n_vars):
                theta = np.hstack([theta, (x[:, i] * x[:, j]).reshape(-1, 1)])
                ind += 1

    # Poly order 3
    if polyorder >= 3:
        for i in range(n_vars):
            for j in range(i, n_vars):
                for k in range(j, n_vars):
                    theta = np.hstack([theta, (x[:, i] * x[:, j] * x[:, k]).reshape(-1, 1)])
                    ind += 1
    
    return theta


# polynomials up to order 3
polyorder = 3 
theta = poolData(x, polyorder)

print("the library matrix theta has", theta.shape[0], "rows (time steps), and", theta.shape[1], "columns (nonlinear library terms).")

print(theta)

In [ ]:
# Perform sparse regression to identify the governing equations
# Using ARD regression from sklearn
reg = linear_model.ARDRegression()
coef1 = reg.fit(theta, dx_dt[:, 0]).coef_
coef2 = reg.fit(theta, dx_dt[:, 1]).coef_
xi = np.vstack([coef1, coef2]).T

print("Identified coefficients:")
print(np.round(xi, 4))

## ---- Implement your own sparse regression method too here ---- ##
# Perform sparse regression to identify the governing equations
# (Your own sparse regression code here)

## ---- Implement your own sparse regression method too here ---- ##

**Can you implement your own model selection algorithm based on Bayesian Evidence?**

In [ ]:
# ------------------------------------------------
# 5 Evaluate Performance of BINDy Model for Forecasting
# ------------------------------------------------

# Define the BINDy ODE function for integration
def bindy_ode(t, x, params):
    # This function needs the 'poolData' and 'xi' from the main script
    theta_ode = poolData(x, params['polyorder'])
    dx = (theta_ode @ params['xi']).flatten()
    return dx

# Parameters for the ODE solver
sindy_params = {'xi': xi, 'polyorder': polyorder}

# test trajectory from new initial condition
x0_test = x_true[-1, :]
t_test = t_eval

# solve the initial value problem
solution = solve_ivp(
    fun=lotka_volterra_system,
    t_span=t_span,
    y0=x0_test,
    args=(params,),
    t_eval=t_test,
    **integrator_keywords
)
x_test_true = solution.y.T

# Simulate the learned model
bindy_solution = solve_ivp(
    fun=bindy_ode,
    t_span=t_span,
    y0=x0_test,
    args=(sindy_params,),
    t_eval=t_eval,
    **integrator_keywords
)
x_bindy_sim = bindy_solution.y.T

# Plot the results and compare
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(t_test, x_test_true[:, 0], 'b', label='True Prey')
plt.plot(t_test, x_bindy_sim[:, 0], 'r--', label='BINDy Prey')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Prey Population: True vs BINDy')
plt.legend()    

plt.subplot(1, 2, 2)
plt.plot(t_test, x_test_true[:, 1], 'b', label='True Predator')
plt.plot(t_test, x_bindy_sim[:, 1], 'r--', label='BINDy Predator')
plt.xlabel('Time')
plt.ylabel('Population')
plt.title('Predator Population: True vs BINDy')
plt.legend()    

# The real challenge
Real-life predator and prey dynamics rarely obey such simple law. Nonethless, the equation captures the essential dynamics. 

The challenge, therefore, is to find better model that is 
- more realistically reflecting reality
- more accurate in forecasting

For example, in the classic Lotka Voltera model, prey are expected to grow exponentially in the absence of predators, unbounded by food. A biologically relevant alternative to the classic Lotka Volterra model is to include a carrying capacity ($Κ$) into the prey population.
$$\frac{dx}{dt}=\alpha x\left(1-\frac{x}{Κ}\right) - \beta x y$$
$$\frac{dy}{dt}=\delta x y - \gamma y$$

This represent's the dynamics of the prey being limited by the constraints in the environment, such that in the absence of predators, the prey population tends to $Κ$, rather than $∞$.

Similarly, the death rate can be regularised:
$$\frac{dx}{dt}=\alpha x\left(1-\frac{x}{Κ}\right) - \beta \frac{x y}{c+x}$$
$$\frac{dy}{dt}=\delta \frac{x y}{c+x} - \gamma y$$

where $c$ is called Michaelis constant, taking inspiration from the [Michaelis-Menten equation](https://en.wikipedia.org/wiki/Michaelis%E2%80%93Menten_kinetics).

### Other things to think about
- The populations can become arbitrarily low and still recover – extinction is impossible
- The equations are deterministic, but the system is inherently stochastic
- Doesn’t consider long-term external factors (e.g. weather and infectious diseases)
- The data is based on pelts trade, which might not always be representative of the real population dynamics.


## Your Project
**Your challenge is to come up with an even "better" model based on real-life data.**
-  You're free to define what is "better" to you - e.g. better forecast? better generalisability? better interpreted? closer to ground truth?
-  How would you evaluate your improvement to the model based on the data?


## About the data
The Hudson Bay Company kept a good record on the number of animal they caught for pelts in Canada between 1845 to 1935, in which the two animals of focus are the lynx (predator) and snowshoe hare (prey). 

To get started, use the [data between 1900-1920](https://jmahaffy.sdsu.edu/courses/f00/math122/labs/labj/lotvol.xls).

If you want more data, here's [another source](http://people.whitman.edu/~hundledr/courses/M250F03/LynxHare.txt) for the year between 1845-1935.

### Some notes about the data
The data is directly measure the animal's population, but the number the company managed to trap for pelts. To use it directly as population data, we are assuming that there is a linear relationship between the pelts trade and the population. How would that affect your modelling?

## Some foreseeable challenges
However, real data is not as friendly to the learning algorithm. 
Foreseeable challenges include:
- Much scarcer data
  - Low Sampling rate
- Inaccuracy in the way $d/dt$ is computed
  - Can we improve it? 
- Much noisier data
  - How to determine the noise level?
- Stochastic noise AND measurement noise
  - How to distinguish between the two?

### Some inspiration for you to get started with the library design
 - https://mc-stan.org/learn-stan/case-studies/lotka-volterra-predator-prey.html
 - https://jckantor.github.io/CBE30338/02.05-Hare-and-Lynx-Population-Dynamics.html

## How you'd be assessed
The main aim of the project is to let you exercise the workflow from data collection, to the design of library, model selection and validation. It is open-ended, so you are not necessarily required to get a very accurate model out of the data. However, it is important that you have demonstrated that you can:
- implement the workflow from data collection to library design and model selection by your own code
- explain the rationale behind the chosen library (e.g. why polynomial library? why not rational/trigonometric functions?)
- justifying the model selected through the Bayesian framework (i.e. why this model and not the others?)
  - in particular, justify the hyperparameters used and how they affect the learning result
- interpret the learning result and the model selected
- test/validate the learning result and the model selected
- communicate well the learning result to others
- show how the current method or learning result can be further improved

